# ABCD Yields

In [1]:
import ROOT as r
import pandas as pd
import numpy as np

In [2]:
def get_abcd(file, BDT_CUT, DNN_CUT):
    rdf = r.RDataFrame("Events", file)
    df = pd.DataFrame(rdf.AsNumpy(["weight", "VBSBDTscore", "abcdnet_score", "Hbbscore", "HighestWjetScore"]))

    HBB_CUT = 0.5
    VQQ_CUT = 0.7 

    df = df.query("Hbbscore > @HBB_CUT & HighestWjetScore > @VQQ_CUT")

    a = df.query("VBSBDTscore > @BDT_CUT & abcdnet_score > @DNN_CUT").weight
    b = df.query("VBSBDTscore > @BDT_CUT & abcdnet_score < @DNN_CUT").weight
    c = df.query("VBSBDTscore < @BDT_CUT & abcdnet_score > @DNN_CUT").weight
    d = df.query("VBSBDTscore < @BDT_CUT & abcdnet_score < @DNN_CUT").weight

    err_a = np.sqrt((a**2).sum())
    err_b = np.sqrt((b**2).sum())
    err_c = np.sqrt((c**2).sum())
    err_d = np.sqrt((d**2).sum())

    a = a.sum()
    b = b.sum()
    c = c.sum()
    d = d.sum()

    if "data_MVA" in file:
        a = np.array(1)

    print(f"Actual A: {a} \\pm {err_a}")
    print(f"Region B: {b} \\pm {err_b}")
    print(f"Region C: {c} \\pm {err_c}")
    print(f"Region D: {d} \\pm {err_d}")

    err_pwhite_a = np.sqrt((err_b * c / d) ** 2 + (err_c * b / d) ** 2 + ((b * c / d**2) * err_d) ** 2)

    print(f"Predicted A: {b * c / d} \\pm {err_pwhite_a}")

    return a, b, c, d

# DATA

In [18]:
data = get_abcd("/data/userdata/aaarora/output/run2/data_MVA_test.root", 0.92, 0.92)

Actual A: 1 \pm 0.0
Region B: 62.0 \pm 7.874007874011811
Region C: 18.0 \pm 4.242640687119285
Region D: 1023.0 \pm 31.984371183438952
Predicted A: 1.0909090909090908 \pm 0.29406446570036243


# SIG

In [19]:
sig = get_abcd("/data/userdata/aaarora/output/run2/sig_MVA_test.root", 0.92, 0.92)

Actual A: 3.7403575143823424 \pm 0.025184301122123446
Region B: 1.007409963695329 \pm 0.01435830749449732
Region C: 0.9248843347709418 \pm 0.01309051327012634
Region D: 0.5316841430162442 \pm 0.010543500259494853
Predicted A: 1.7524270873083125 \pm 0.04946409684225818


# BKG

In [20]:
bkg = get_abcd("/data/userdata/aaarora/output/run2/bkg_MVA_test.root", 0.92, 0.92)

Actual A: 1.1911118952802233 \pm 0.3042425076947078
Region B: 59.095666535109714 \pm 3.4334041371607897
Region C: 18.07647297522063 \pm 1.6587443971467315
Region D: 965.5325179869376 \pm 10.704935611653111
Predicted A: 1.106375185894063 \pm 0.12078660331565702
